In [ ]:
## Imports
# import os
from pathlib import Path
# import shutil
import warnings
import opendatasets as od
from typing import Optional, Callable, Tuple, Dict
import numpy as np
import matplotlib.pyplot as plt
import torch
# import torchvision
from torch.utils.data import random_split
from torchvision.datasets.vision import VisionDataset

In [ ]:
## Simulate the args like in the `main_*.py` files
class ARGS:
    num_users:int = 100

args = ARGS()
args.num_users

In [ ]:
## Define custom CobaDataset class
class COBA(VisionDataset):
    """
    `COBA <https://www.kaggle.com/datasets/earltankardjr/coba-iobt-dataset> Dataset`
    Args:
        root (string): Root directory of dataset where ``COBA/raw/data.pt``
            and  ``COBA/raw/targets.pt`` exist.
        train (bool, optional): If `True`, creates dataset from ``training.pt``,
            otherwise from ``test.pt``. -- This currently doesn't do anything!
        download (bool, optional): If `True`, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """
    resources:list = ["https://www.kaggle.com/datasets/earltankardjr/coba-iobt-dataset"] # link(s) to coba dataset
    training_file:str = "training.pt"
    test_file:str = "test.pt"
    # training_file:str = "training.pt"
    # test_file:str = "test.pt"
    classes:list = ["0 - airplane",
                    "1 - ambulance",
                    "2 - briefcase",
                    "3 - cannon",
                    "4 - car",
                    "5 - civilian",
                    "6 - dagger",
                    "7 - dog",
                    "8 - handgun",
                    "9 - missilerocket",
                    "10 - rifle",
                    "11 - soldier",
                    "12 - tank",
                    "13 - truck"
    ]

    
    @property
    def train_labels(self) -> torch.Tensor:
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self) -> torch.Tensor:
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self) -> torch.Tensor:
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self) -> torch.Tensor:
        warnings.warn("test_data has been renamed data")
        return self.data

    def __init__(self, root:str, train:bool=True, transform:Optional[Callable]=None, target_transform: Optional[Callable]=None, download:bool=False) -> None:
        super().__init__(root, transform=transform, target_transform=target_transform)
        self.train = train

        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +
                               ' You can use download=True to download it')

        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file
        
        self.data, self.targets = self._load_data()


    def download(self) -> None:
        if self._check_exists():
            return

        # Make Raw directory
        Path.mkdir(self.raw_folder, exist_ok=True, parents=True)

        # Download dataset file
        coba_dataset_url:str = self.resources[0]
        od.download(coba_dataset_url)

        ## Move dataset
        coba_iobt_npy_file:str = "iobt_128_128.npy"

        downloaded_coba_dataset_path = Path.cwd().joinpath("coba-iobt-dataset", coba_iobt_npy_file)
        
        if not downloaded_coba_dataset_path.exists():
            raise Exception("Failed to download and locate COBA dataset")
        
        new_coba_dataset_path = Path.cwd().joinpath(self.raw_folder, coba_iobt_npy_file)
        downloaded_coba_dataset_path.rename(new_coba_dataset_path)
        
        ## Format dataset
        with open(new_coba_dataset_path, "rb") as f:
            data = torch.tensor(np.load(f), dtype=torch.float32)
            targets = torch.tensor(np.load(f, allow_pickle=True), dtype=torch.int64)

        ## Save dataset
        torch.save(data, self.raw_folder.joinpath("data.pt"))
        torch.save(targets, self.raw_folder.joinpath("targets.pt"))
        
        ## Delete unnecessary files/directories
        Path.rmdir(Path.cwd().joinpath("coba-iobt-dataset"))
        Path.unlink(Path.cwd().joinpath(self.raw_folder, coba_iobt_npy_file))

    def _load_data(self) -> Tuple[torch.Tensor, torch.Tensor]:
        image_file:str = "data.pt"
        data = torch.load(Path.cwd().joinpath(self.raw_folder, image_file))
        
        label_file:str = "targets.pt"
        targets = torch.load(Path.cwd().joinpath(self.raw_folder, label_file))

        return data, targets


    def __len__(self) -> int:
        return len(self.data)
        # return len(self.labels)

    def __getitem__(self, idx:int) -> Tuple[torch.Tensor, torch.Tensor]:
        image = self.data[idx]
        label = self.targets[idx]
        # label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            target = self.target_transform(target)
        
        return image, label

    @property
    def raw_folder(self) -> Path:
        return Path(self.root, self.__class__.__name__, "raw")

    @property
    def processed_folder(self) -> Path:
        return Path(self.root, self.__class__.__name__, "processed")

    @property
    def class_to_idx(self) -> Dict[int, str]:
        # class_dict:dict[int,str] = {int(label.replace(" ","").split("-")[0]): label.replace(" ","").split("-")[1] for label in self.classes} #one liner
        class_dict:Dict[int,str] = {}
        for label in self.classes:
            encoded_val, name = label.replace(" ","").split("-")
            class_dict[int(encoded_val)] = name
        return class_dict

    def _check_exists(self) -> bool:
        return Path.exists(Path(self.raw_folder, "data.pt")) and Path.exists(Path(self.raw_folder, "targets.pt"))
        # return (Path.exists(Path.joinpath(self.raw_folder, self.training_file)) and Path.exists(Path.joinpath(self.raw_folder, self.test_file)))

In [ ]:
## Initialize CobaDataset
coba_dataset = COBA(root="data/coba", download=True)

In [ ]:
## Create training and testing data -- method 1
train_size = int(0.8 * len(coba_dataset))
test_size = len(coba_dataset) - train_size
train_dataset, test_dataset = random_split(dataset=coba_dataset, lengths=[train_size, test_size])

In [ ]:
len(train_dataset.indices)

In [ ]:
## Plot random train sample
example_image = coba_dataset[np.random.choice(train_dataset.indices, 1).item()]
img, label = example_image
label_encodings = train_dataset.dataset.class_to_idx

print(f"Label: {label_encodings[label.item()]}")
plt.imshow(img)

In [ ]:
## The random split works!
dups = 0
for index in test_dataset.indices:
    if index in train_dataset.indices:
        dups += 1
print(dups)        

In [ ]:
# Try iid example
def iid(dataset, num_users):
    """
    Sample I.I.D. client data from MNIST dataset
    :param dataset:
    :param num_users:
    :return: dict of image index
    """
    num_items = int(len(dataset)/num_users)
    dict_users, all_idxs = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        dict_users[i] = set(np.random.choice(all_idxs, num_items, replace=False))
        all_idxs = list(set(all_idxs) - dict_users[i])
    return dict_users

In [ ]:
dict_users_train = iid(dataset=train_dataset.dataset, num_users=100)
for user,d in dict_users_train.items():
    print(f"user:{user}\t\t len:{len(d)}")

In [ ]:
## Try noniid example
import random

def noniid(dataset, num_users, shard_per_user, rand_set_all=[]):
    """
    Sample non-I.I.D client data from MNIST dataset
    :param dataset:
    :param num_users:
    :return:
    """
    dict_users = {i: np.array([], dtype='int64') for i in range(num_users)}

    idxs_dict = {}
    for i in range(len(dataset)):
        # label = torch.tensor(dataset.targets[i]).item()
        label = dataset.targets[i].item()
        if label not in idxs_dict.keys():
            idxs_dict[label] = []
        idxs_dict[label].append(i)

    num_classes = len(np.unique(dataset.targets))
    shard_per_class = int(shard_per_user * num_users / num_classes)

    #debug
    print(f"num_classes: {num_classes}")
    print(f"shard_per_class: {shard_per_class}")
    
    for label in idxs_dict.keys():
        x = idxs_dict[label]
        num_leftover = len(x) % shard_per_class
        leftover = x[-num_leftover:] if num_leftover > 0 else []
        x = np.array(x[:-num_leftover]) if num_leftover > 0 else np.array(x)
        x = x.reshape((shard_per_class, -1))
        x = list(x)

        #debug
        # print(f"x: {x}")

        for i, idx in enumerate(leftover):
            x[i] = np.concatenate([x[i], [idx]])
        idxs_dict[label] = x

        #debug
        # print(f"idxs_dict: {idxs_dict}")

    if len(rand_set_all) == 0:
        rand_set_all = list(range(num_classes)) * shard_per_class
        random.shuffle(rand_set_all)
        
        #debug
        print(f"len(rand_set_all) = {len(rand_set_all)}")

        try:
            rand_set_all = np.array(rand_set_all).reshape((num_users, -1))
        except ValueError as ve:
            print(f"ValueError: {ve}\n\nAttempting to reshape...")
            for i in range(num_users,0,-1):
                try:
                    rand_set_all = np.array(rand_set_all).reshape((i, -1))
                    print(f"New num_users: {i}")
                    global args
                    args.num_users = i
                    num_users = i
                    break
                except ValueError:
                    continue
        print(f"rand_set_all.shape: {rand_set_all.shape}")
    
    # divide and assign
    for i in range(num_users):
        rand_set_label = rand_set_all[i]
        rand_set = []
        for label in rand_set_label:
            idx = np.random.choice(len(idxs_dict[label]), replace=False)
            rand_set.append(idxs_dict[label].pop(idx))
        dict_users[i] = np.concatenate(rand_set)

    dict_users = {key: val for key,val in dict_users.items() if len(val)}
    
    test = []
    for key, value in dict_users.items():
        # x = np.unique(torch.tensor(dataset.targets)[value])
        x = np.unique(dataset.targets[value])
        assert(len(x)) <= shard_per_user
        test.append(value)
    test = np.concatenate(test)
    assert(len(test) == len(dataset))
    assert(len(set(list(test))) == len(dataset))

    return dict_users, rand_set_all

In [ ]:
dict_users_train, rand_set_all = noniid(dataset=train_dataset.dataset, num_users=100, shard_per_user=2)
for user,d in dict_users_train.items():
    print(f"user:{user}\t\t len:{len(d)}")

In [ ]:
args.num_users

In [ ]:
coba_dataset.class_to_idx

In [ ]:
import copy
import pickle
import numpy as np
import pandas as pd
import torch

from utils.options import args_parser
from utils.train_utils import get_data, get_model
from models.Update import LocalUpdate
from models.test import test_img
import os
## Simulate the args like in the `main_*.py` files
class ARGS:
    #federated arguments
    epochs:int = 1000         # rounds of training
    num_users:int = 100       # number of users: K
    shard_per_user:int = 2    # classes per user
    frac:float = 0.1          # the fraction of clients: C
    local_ep:int = 1          # the number of local epochs: E
    local_bs:int = 10         # local batch size: B
    bs:int = 128              # test batch size
    lr:float = 0.01           # learning rate
    # results_save:str = "run1"
    momentum:float = 0.5      # SGD momentum (default: 0.5)
    # gpu:int = 0
    split:str = "user"        # train-test split type, user or sample
    # grad_norm:str           # use_gradnorm_avging
    local_ep_pretrain:int = 0 # the number of pretrain local ep
    lr_decay:float = 1.0      # learning rate decay per round

    # model arguments
    model:str = "cnn"          # model name
    kernel_num:int = 9         # number of each kind of kernel
    kernel_sizes:str = "3,4,5" # comma-separated kernel size to use for convolution
    norm:str = "batch_norm"    # batch_norm, layer_norm, or None
    num_filters:int = 32       # number of filters for conv nets
    max_pool:str = True        # whether use max pooling rather than strided convolutions
    num_layers_keep:int = 1    # number layers to keep
    
    # other arguments
    dataset:str = "coba"     # name of dataset
    iid:bool = True          # "store_true" #whether iid or not
    num_classes:int = 14     # number of classes
    num_channels:int = 3     # number of channels of images RGB
    gpu:int = 0              # GPU ID, -1 for CPU
    stopping_rounds:int = 10 # rounds of early stopping
    verbose:bool = True      # "store_true"
    print_freq:int = 100     # print loss frequency during training
    seed:int = 1             # random seed (default:1)
    test_freq:int = 1        # how often to test on val set
    load_fed:str = ""        # define pretrained federated model path
    results_save:str = "/"   # define fed results save folder
    start_saving:int = 0     # when to start saving models


args = ARGS()
args.num_users

In [ ]:
args.device = torch.device(
        "cuda:{}".format(args.gpu)
        if torch.cuda.is_available() and args.gpu != -1
        else "cpu"
)
args.device

In [ ]:
dataset_train, dataset_test, dict_users_train, dict_users_test = get_data(args)

if args.dataset == "coba":
    dataset_train, dataset_test = dataset_train.dataset, dataset_test.dataset

base_dir = "./save/{}/{}_iid{}_num{}_C{}_le{}/shard{}/{}/".format(
    args.dataset,
    args.model,
    args.iid,
    args.num_users,
    args.frac,
    args.local_ep,
    args.shard_per_user,
    args.results_save,
)
if not os.path.exists(os.path.join(base_dir, "fed")):
    os.makedirs(os.path.join(base_dir, "fed"), exist_ok=True)

dict_save_path = os.path.join(base_dir, "dict_users.pkl")
with open(dict_save_path, "wb") as handle:
    pickle.dump((dict_users_train, dict_users_test), handle)

# build model
net_glob = get_model(args)
net_glob.train()

# training
results_save_path = os.path.join(base_dir, "fed/results.csv")

loss_train = []
net_best = None
best_loss = None
best_acc = None
best_epoch = None

lr = args.lr
results = []

for _iter in range(args.epochs):
    w_glob = None
    loss_locals = []
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    print("Round {}, lr: {:.6f}, {}".format(_iter, lr, idxs_users))

    for idx in idxs_users:
        local = LocalUpdate(
            args=args, dataset=dataset_train, idxs=dict_users_train[idx]
        )
        net_local = copy.deepcopy(net_glob)

        w_local, loss = local.train(net=net_local.to(args.device))
        loss_locals.append(copy.deepcopy(loss))

        if w_glob is None:
            w_glob = copy.deepcopy(w_local)
        else:
            for k in w_glob.keys():
                w_glob[k] += w_local[k]

    lr *= args.lr_decay

    # update global weights
    for k in w_glob.keys():
        w_glob[k] = torch.div(w_glob[k], m)

    # copy weight to net_glob
    net_glob.load_state_dict(w_glob)

    # print loss
    loss_avg = sum(loss_locals) / len(loss_locals)
    loss_train.append(loss_avg)

    if (_iter + 1) % args.test_freq == 0:
        net_glob.eval()

        # pylint: disable=unbalanced-tuple-unpacking
        acc_test, loss_test = test_img(net_glob, dataset_test, args)
        print(
            "Round {:3d}, Average loss {:.3f}, Test loss {:.3f}, Test accuracy: {:.2f}".format(
                _iter, loss_avg, loss_test, acc_test
            )
        )

        if best_acc is None or acc_test > best_acc:
            net_best = copy.deepcopy(net_glob)
            best_acc = acc_test
            best_epoch = _iter

        # if (iter + 1) > args.start_saving:
        #     model_save_path = os.path.join(base_dir, 'fed/model_{}.pt'.format(_iter + 1))
        #     torch.save(net_glob.state_dict(), model_save_path)

        results.append(np.array([_iter, loss_avg, loss_test, acc_test, best_acc]))
        final_results = np.array(results)
        final_results = pd.DataFrame(
            final_results,
            columns=["epoch", "loss_avg", "loss_test", "acc_test", "best_acc"],
        )
        final_results.to_csv(results_save_path, index=False)

    if (_iter + 1) % 50 == 0:
        best_save_path = os.path.join(base_dir, "fed/best_{}.pt".format(_iter + 1))
        model_save_path = os.path.join(
            base_dir, "fed/model_{}.pt".format(_iter + 1)
        )
        torch.save(net_best.state_dict(), best_save_path)
        torch.save(net_glob.state_dict(), model_save_path)

print("Best model, iter: {}, acc: {}".format(best_epoch, best_acc))

In [ ]:
## Troubleshooting with Shape errors
# image,label = coba_dataset[0]
# print(image.shape)
# image = image.permute(2,0,1)
# print(image.shape)